In [1]:
import json
import pandas as pd
from pandas import json_normalize

# Load the JSON data
with open('../../data/masterjson.json', 'r') as file:
    data = json.load(file)


FileNotFoundError: [Errno 2] No such file or directory: '../../data/masterjson.json'

In [3]:
for item in data['results']:
    # Merge the attributes from 'attrs' into the main item
    item.update(item['attrs'])
    # Delete the now unnecessary keys
    del item['attrs']
    del item['id']
    del item['weight']
    del item['geom_quadindex']
    del item['geom_st_box2d']
    del item['origin']
    del item['rank']
    del item['x']
    del item['y']
    del item['zoomlevel']
    del item['num']
    del item['detail']

In [4]:
nodes_data = data['results']

# Convert to DataFrame
df = pd.DataFrame(nodes_data)

print(df.columns)


Index(['label', 'lat', 'lon', 'objectclass', 'Kategorie', 'schoolshort',
       'name', 'quarter', 'city', 'address', 'zipcode'],
      dtype='object')


In [5]:
data_dict = df.to_dict(orient='records')



In [6]:
### nur als proof of concept kann übersprungen werdem
import pandas as pd

def finde_gemeinde(plz):
    dateiname = 'PLZO_CSV_LV95.csv'
    df = pd.read_csv(dateiname, encoding='utf-8', delimiter=';')

    # Ensure PLZ column is of type int
    df['PLZ'] = df['PLZ'].astype(str).str.strip().astype(int)
    
    matching_rows = df[df['PLZ'] == plz]['Gemeindename']

    if matching_rows.empty:
        return "Unbekannt"

    return matching_rows.values[0]

# Test der Funktion
postleitzahl = 1702
print(f"Die Gemeinde für die Postleitzahl {postleitzahl} ist {finde_gemeinde(postleitzahl)}.")



Die Gemeinde für die Postleitzahl 1702 ist Unbekannt.


In [7]:
import pandas as pd
import json

def finde_gemeinde(plz):
    """
    Diese Funktion sucht nach einer Gemeinde basierend auf einer gegebenen Postleitzahl (PLZ).

    :param plz: Die Postleitzahl, für die die Gemeinde gesucht werden soll.
    :return: Den Namen der Gemeinde, die der gegebenen PLZ entspricht, oder None, wenn keine gefunden wird.
    """

    # Dateiname der CSV-Datei, die die PLZ und Gemeindenamen enthält.
    dateiname = 'PLZO_CSV_LV95.csv'

    # Die CSV-Datei wird in einen Pandas DataFrame geladen.
    # Das Einlesen der Datei bei jedem Aufruf ist nicht besonders effizient, aber die Datenmenge ist wahrscheinlich zu gering, um einen spürbaren Unterschied zu machen.
    df = pd.read_csv(dateiname, encoding='utf-8', delimiter=';')

    # Suche nach der Gemeinde, die der gegebenen PLZ entspricht.
    gemeinde = df[df['PLZ'] == int(plz)]['Gemeindename'].values

    # Wenn eine Gemeinde gefunden wird, wird ihr Name zurückgegeben.
    if len(gemeinde) > 0:
        return gemeinde[0]
    # Wenn keine Gemeinde gefunden wird, wird None zurückgegeben.
    else:
        return None

# Das ursprüngliche Dictionary mit den Hochschuldaten.
colleges = data_dict

# Eine leere Liste, um die aktualisierten Hochschuldaten zu speichern.
updated_colleges = []

# Durch jeden Eintrag im colleges-Dictionary iterieren.
for college in colleges:
    # Überprüfen, ob der Stadtnamen "Unknown City" hat.
    if college['city'] == "Unknown City":
        plz = college['zipcode']
        # Die Gemeinde basierend auf der PLZ suchen.
        city_name = finde_gemeinde(plz)
        # Wenn eine gültige Gemeinde gefunden wird, wird der Stadtnamen im college-Eintrag aktualisiert und der aktualisierte Eintrag wird der updated_colleges-Liste hinzugefügt.
        if city_name:
            college['city'] = city_name
            updated_colleges.append(college)
    # Wenn der Stadtnamen bereits bekannt ist, wird der Eintrag unverändert der updated_colleges-Liste hinzugefügt.
    else:
        updated_colleges.append(college)

# Das aktualisierte Dictionary wird in data_dict gespeichert.
data_dict = updated_colleges



In [8]:

colleges_clean = data_dict
# Zählen, wie oft "Unknown City" vorkommt
unknown_city_count = sum(1 for college in colleges_clean if college['city'] == "Unknown City")

print(f"'Unknown City' erscheint {unknown_city_count} Mal in der colleges_clean.json Datei.")


'Unknown City' erscheint 0 Mal in der colleges_clean.json Datei.


In [9]:
import json

def simplify_string(s):
    """
    Diese Funktion nimmt einen String `s` entgegen und entfernt bestimmte HTML-Tags, um den String zu vereinfachen.

    :param s: Der Eingabe-String, der bearbeitet werden soll.
    :return: Ein vereinfachter String ohne bestimmte HTML-Tags.
    """

    # Den Startindex für den zu entfernenden Bereich finden. Dieser liegt direkt nach dem ersten "<i>"-Tag.
    start_index = s.find('<i>') + 3

    # Den Endindex für den zu entfernenden Bereich finden. Dieser liegt direkt vor dem ersten "</i>"-Tag.
    end_index = s.find('</i>')

    # Den Bereich zwischen den "<i>" und "</i>"-Tags aus dem String entfernen.
    result = s.replace(s[start_index:end_index], '')

    # Alle "<b>" und "</b>"-Tags aus dem resultierenden String entfernen.
    result = result.replace('<b>', '').replace('</b>', '').replace('<i>', '').replace('</i>','')

    # Den bereinigten String zurückgeben, nachdem überflüssige Leerzeichen entfernt wurden.
    return result.strip()

def clean_labels_in_json(data_dict, file_path=None):
    """
    Diese Funktion nimmt ein Dictionary `data_dict` oder einen Dateipfad `file_path` zu einer JSON-Datei entgegen und
    bereinigt die 'label'-Einträge in jedem Element des JSON.

    :param data_dict: Das Eingabe-Dictionary, das bearbeitet werden soll.
    :param file_path: Der Pfad zur JSON-Datei, die bearbeitet werden soll.
    :return: Ein bereinigtes Dictionary.
    """

    # Wenn ein Dateipfad angegeben wurde, wird die JSON-Datei geöffnet und als Dictionary geladen.
    if file_path is not None:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
    else:
        # Wenn kein Dateipfad angegeben wurde, wird das übergebene Dictionary verwendet.
        data = data_dict

    # Über jedes Element im Dictionary iterieren.
    for item in data:
        # Den 'label'-Eintrag des aktuellen Elements mit der `simplify_string`-Funktion bereinigen.
        item['label'] = simplify_string(item['label'])

    # Das bereinigte Dictionary zurückgeben.
    return data

# Die `clean_labels_in_json`-Funktion wird auf das `data_dict` angewendet, um die 'label'-Einträge zu bereinigen.
data_dict = clean_labels_in_json(data_dict)


In [10]:
# Specify the path where you want to save the JSON file
output_path = '../../data/colleges_clean.json'

# Save the dictionary as a JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(data_dict, file, indent=4, ensure_ascii=False)

In [ ]:
## hier wird die weitere verarbeitung umgesetzt (anpassung der anzahl colleges etc)

In [2]:
import json

# Load the JSON file
with open('colleges_clean.json', 'r') as file:
    data = json.load(file)

# You can now use the 'data' variable which contains the JSON data


In [3]:
i = 0
for entry in data:
    print(entry['label'])
    i+=1
    #print(entry)
print(i)

Université de Genève UNIGE, Pignon (GE) - Genève
Université de Genève UNIGE, Ancienne École de Médecine (GE) - Genève
Université de Genève, Valais Campus Sion (VS) - Sion
Université de Genève UNIGE, Sciences (GE) - Genève
Université de Genève UNIGE, Saint-Ours (GE) - Genève
Université de Genève UNIGE, Observatoire (GE) - Versoix
Université de Genève UNIGE, Maraîchers (GE) - Genève
Université de Genève UNIGE, Ecogia (GE) - Versoix
Université de Genève UNIGE (GE) - Genève
Université de Genève UNIGE, Landolt/Mirabeau (GE) - Genève
Université de Genève UNIGE, Comédie (GE) - Genève
Université de Genève UNIGE, Battelle (GE) - Carouge (GE)
Université de Genève UNIGE, Dufour (GE) - Genève
Université de Genève UNIGE, Philosophes (GE) - Genève
Université de Genève UNIGE, Carl Vogt (GE) - Genève
Université de Genève UNIGE, S.I.P. (GE) - Genève
Université de Fribourg UNIFR, Rue Pierre-Aeby (FR) - Fribourg
Université de Fribourg UNIFR, Route des Bonnesfontaines (FR) - Fribourg
Université de Fribour

In [4]:
location_words = [
    "Standort",  # German for location/site
    "Site",      # French for location/site
    "Sede",      # Italian for location/site
    "Campus"     # Common in all three languages
]
zurich_universities = [
    "ZHAW",  # Zürcher Hochschule für Angewandte Wissenschaften
    "UZH",   # Universität Zürich
    "ETHZ",  # Eidgenössische Technische Hochschule Zürich
    "ZHDK"   # Zürcher Hochschule der Künste
]



In [7]:
#Zeigt alle hochschulen mit mehr alks einem standort pro plz

In [6]:

from collections import defaultdict

# Erstellen eines Dictionaries, um die Anzahl der Einträge pro Hochschule und Postleitzahl zu zählen
counts = defaultdict(lambda: defaultdict(int))

# Durchlaufen der Daten und Zählen der Einträge
for entry in data:
    school = entry['schoolshort']
    zipcode = entry['zipcode']
    counts[school][zipcode] += 1

# Anzeigen der Einträge, die mehr als einen Eintrag pro Hochschule und Postleitzahl haben
i =0
for school, zipcodes in counts.items():
    for zipcode, count in zipcodes.items():
        if count > 1:
            print(f"Hochschule: {school}, Postleitzahl: {zipcode}, Anzahl der Einträge: {count}")
            i+=1
            
            
            
print(i)


Hochschule: UNIGE, Postleitzahl: 1200, Anzahl der Einträge: 3
Hochschule: UNIGE, Postleitzahl: 1205, Anzahl der Einträge: 7
Hochschule: UNIGE, Postleitzahl: 1290, Anzahl der Einträge: 2
Hochschule: UNIFRI, Postleitzahl: 1700, Anzahl der Einträge: 9
Hochschule: UNIL, Postleitzahl: 1011, Anzahl der Einträge: 5
Hochschule: UNIBAS, Postleitzahl: 4010, Anzahl der Einträge: 16
Hochschule: UNIBAS, Postleitzahl: 4058, Anzahl der Einträge: 2
Hochschule: UNIBAS, Postleitzahl: 4051, Anzahl der Einträge: 14
Hochschule: UNIBAS, Postleitzahl: 4055, Anzahl der Einträge: 3
Hochschule: UNIBAS, Postleitzahl: 4031, Anzahl der Einträge: 4
Hochschule: UNIBAS, Postleitzahl: 4056, Anzahl der Einträge: 5
Hochschule: UNIBAS, Postleitzahl: 4012, Anzahl der Einträge: 2
Hochschule: EPFL, Postleitzahl: 1024, Anzahl der Einträge: 4
Hochschule: ETHZ, Postleitzahl: 8006, Anzahl der Einträge: 16
Hochschule: ETHZ, Postleitzahl: 8001, Anzahl der Einträge: 4
Hochschule: ETHZ, Postleitzahl: 8092, Anzahl der Einträge: 2
Ho

In [7]:
#now we visualize this on a map
import folium



# Create a map object, centered at an average location
map_object = folium.Map(location=[46.5, 6.6], zoom_start=10)

# Iterate through each entry in the data
for item in data:
    # Extract latitude, longitude, and other information
    lat = item['lat']
    lon = item['lon']
    name = item['name']
    address = item['address']
    label = item['label']

    # Create a popup string with the desired information
    popup_string = f"{name}<br>{address}<br>{label}"

    # Add a marker to the map
    folium.Marker([lat, lon], popup=popup_string).add_to(map_object)

# Display the map in a Jupyter Notebook
map_object




In [8]:
from collections import defaultdict

# Erstellen eines Dictionaries, um die Anzahl identischer Label pro Hochschule und Postleitzahl zu zählen
counts_labels = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

# Durchlaufen der Daten und Zählen identischer Label
for entry in data:
    school = entry['schoolshort']
    zipcode = entry['zipcode']
    label = entry['label']
    counts_labels[school][zipcode][label] += 1

# Anzeigen der identischen Label pro Hochschule und Postleitzahl und deren Zählung
identical_label_count = 0
for school, zipcodes in counts_labels.items():
    for zipcode, labels in zipcodes.items():
        for label, count in labels.items():
            if count > 1:
                print(f"Hochschule: {school}, Postleitzahl: {zipcode}, Label: {label}, Anzahl identischer Einträge: {count}")
                identical_label_count += 1

print(identical_label_count)


Hochschule: UNIFRI, Postleitzahl: 1700, Label: Université de Fribourg UNIFR, Site Beauregard (FR) - Fribourg, Anzahl identischer Einträge: 2
Hochschule: UNIL, Postleitzahl: 1011, Label: Université de Lausanne UNIL, Site Bugnon CHUV (VD) - Lausanne, Anzahl identischer Einträge: 5
Hochschule: UNIBAS, Postleitzahl: 4010, Label: Universität Basel, Standort Bernoullistrasse (BS) - Basel, Anzahl identischer Einträge: 3
Hochschule: UNIBAS, Postleitzahl: 4010, Label: Universität Basel, Standort Hebelstrasse (BS) - Basel, Anzahl identischer Einträge: 2
Hochschule: UNIBAS, Postleitzahl: 4010, Label: Universität Basel, Standort Klingelbergstrasse (BS) - Basel, Anzahl identischer Einträge: 2
Hochschule: UNIBAS, Postleitzahl: 4010, Label: Universität Basel, Standort Spitalstrasse (BS) - Basel, Anzahl identischer Einträge: 3
Hochschule: UNIBAS, Postleitzahl: 4051, Label: Universität Basel, Standort Steinengraben (BS) - Basel, Anzahl identischer Einträge: 2
Hochschule: UNIBAS, Postleitzahl: 4051, Lab

In [9]:
# Funktion zum Entfernen von Duplikaten
def remove_duplicates(entries, exclude_list=None):
    unique_entries = []
    seen_labels = set()
    for entry in entries:
        if exclude_list and entry['schoolshort'] in exclude_list:
            unique_entries.append(entry)
        else:
            key = (entry['schoolshort'], entry['zipcode'], entry['label'])
            if key not in seen_labels:
                unique_entries.append(entry)
                seen_labels.add(key)
    return unique_entries

# Entfernen aller Duplikate
cleaned_data = remove_duplicates(data)

# Speichern als JSON-Datei
with open('cleaned_data.json', 'w') as file:
    json.dump(cleaned_data, file)

# Entfernen aller Duplikate, außer die aus Zürcher Universitäten
cleaned_data_except_zurich = remove_duplicates(data, zurich_universities)

# Speichern als JSON-Datei
with open('cleaned_data_except_zurich.json', 'w') as file:
    json.dump(cleaned_data_except_zurich, file)

In [10]:
#now we visualize this on a map
import folium



# Create a map object, centered at an average location
map_object = folium.Map(location=[46.5, 6.6], zoom_start=10)

# Iterate through each entry in the data
for item in cleaned_data:
    # Extract latitude, longitude, and other information
    lat = item['lat']
    lon = item['lon']
    name = item['name']
    address = item['address']
    label = item['label']

    # Create a popup string with the desired information
    popup_string = f"{name}<br>{address}<br>{label}"

    # Add a marker to the map
    folium.Marker([lat, lon], popup=popup_string).add_to(map_object)

# Display the map in a Jupyter Notebook
map_object


In [ ]:
currentdata = data #backup


In [ ]:
data = currentdata

In [19]:
# Einlesen der Datei 'cleaned_data.json'
with open('cleaned_data.json', 'r') as file:
        data = json.load(file)

In [ ]:
# Einlesen der Datei 'cleaned_data_except_zurich.json'
with open('cleaned_data_except_zurich.json', 'r') as file:
     data = json.load(file)

In [53]:
from collections import Counter
kuerzel_count = Counter(entry['schoolshort'] for entry in data)

kuerzel_count

Counter({'UNIGE': 16,
         'UNIFRI': 9,
         'UNIL': 8,
         'UNIBAS': 40,
         'EPFL': 7,
         'ETHZ': 6,
         'UNILU': 1,
         'HSG': 15,
         'UNINE': 5,
         'UNIBE': 9,
         'UZH': 4,
         'USI': 2,
         'PHSH': 1,
         'PHLU': 6,
         'BEJUNE': 3,
         'HEPFRI': 2,
         'PHGR': 1,
         'PHZG': 1,
         'PHBE': 5,
         'PHTG': 1,
         'PHVS': 1,
         'PHSG': 3,
         'PHZH': 1,
         'HEPVD': 2,
         'ZHAW': 5,
         'HSLU': 8,
         'KALAI': 3,
         'HESSO': 38,
         'OST': 2,
         'SUPSI': 5,
         'FHBE': 20,
         'ZHDK': 1,
         'FHGR': 1})

In [18]:
# Angenommenes Kürzel 'schoolshort' (Beispiel)
selected_schoolshort = "UZH"

# Filtern und Anzeigen des Labels und der Postleitzahl von jedem Eintrag mit dem ausgewählten Kürzel
filtered_entries = [(entry['label'], entry['zipcode']) for entry in data if entry['schoolshort'] == selected_schoolshort]

filtered_entries


[('Universität Zürich UZH (ZH) - Zürich', '8001'),
 ('Universität Zürich UZH, Standort Zentrum (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH (ZH) - Zürich', '8006'),
 ('Universität Zürich UZH (ZH) - Zürich', '8044'),
 ('Universität Zürich UZH (ZH) - Zürich', '8032'),
 ('Universität Zürich UZH (ZH) - Zürich', '8006'),
 ('Universität Zürich UZH (ZH) - Zürich', '8032'),
 ('Universität Zürich UZH, Zentralbibliothek (ZH) - Zürich', '8001'),
 ('Universität Zürich UZH (ZH) - Zürich', '8024'),
 ('Universität Zürich UZH (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH, Standort Oerlikon (ZH) - Zürich', '8050'),
 ('Universität Zürich UZH, Standort Zentrum (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH (ZH) - Zürich', '8006'),
 ('Universität Zürich UZH (ZH) - Zürich', '8005'),
 ('Universität Zürich UZH (ZH) - Zürich', '8001'),
 ('Universität Zürich UZH (ZH) - Zürich', '8006'),
 ('Universität Zürich UZH (ZH) - Zürich', '8001'),
 ('Unive

In [12]:
# Löschen von Einträgen mit dem Kürzel 'ETHZ', die nicht eine der bestimmten Postleitzahlen haben
allowed_zipcodes = {'8603', '8091', '8093', '8008', '8050'}

# Filtern der Daten
filtered_data = [entry for entry in data if not (entry['schoolshort'] == 'ETHZ' and entry['zipcode'] not in allowed_zipcodes)]

filtered_data


[{'label': 'Université de Genève UNIGE, Pignon (GE) - Genève',
  'lat': 46.194271087646484,
  'lon': 6.140400409698486,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Université de Genève - Uni Pignon',
  'quarter': 'Jonction',
  'city': 'Genève',
  'address': "Boulevard du Pont-d'Arve, 1200, Genève",
  'zipcode': '1200'},
 {'label': 'Université de Genève UNIGE, Ancienne École de Médecine (GE) - Genève',
  'lat': 46.19590759277344,
  'lon': 6.136425971984863,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Unknown Amenity',
  'quarter': 'Jonction',
  'city': 'Genève',
  'address': "Rue de l'Ecole-de-Médecine, 1205, Genève",
  'zipcode': '1205'},
 {'label': 'Université de Genève, Valais Campus Sion (VS) - Sion',
  'lat': 46.22985076904297,
  'lon': 7.398563861846924,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Fondation universitaire Kurt B

In [16]:
#hier immer eintragen PHZH,UZH;ETHZ und ZHAW, 

selected_schoolshort = "UZH"

# Filtern und Anzeigen des Labels und der Postleitzahl von jedem Eintrag mit dem ausgewählten Kürzel
filtered_entries = [(entry['label'], entry['zipcode']) for entry in filtered_data if entry['schoolshort'] == selected_schoolshort]

filtered_entries

[('Universität Zürich UZH, Standort Zentrum (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH, Standort Oerlikon (ZH) - Zürich', '8050'),
 ('Universität Zürich UZH, Standort Zentrum (ZH) - Zürich', '8091'),
 ('Universität Zürich UZH, Standort Kilchberg (ZH) - Zürichsee (ZH),Kilchberg (ZH)',
  '8802'),
 ('Universität Zürich UZH, Standort Zentrum (ZH) - Zürich', '8032'),
 ('Universität Zürich UZH, Standort Oerlikon (ZH) - Zürich', '8050'),
 ('Universität Zürich UZH, Standort Oerlikon (ZH) - Zürich', '8050')]

In [14]:
data = filtered_data

In [15]:
# Filtern der Daten
filtered_data = [entry for entry in filtered_data if not (entry['schoolshort'] == 'UZH' and 'Standort' not in entry['label'])]

filtered_data

[{'label': 'Université de Genève UNIGE, Pignon (GE) - Genève',
  'lat': 46.194271087646484,
  'lon': 6.140400409698486,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Université de Genève - Uni Pignon',
  'quarter': 'Jonction',
  'city': 'Genève',
  'address': "Boulevard du Pont-d'Arve, 1200, Genève",
  'zipcode': '1200'},
 {'label': 'Université de Genève UNIGE, Ancienne École de Médecine (GE) - Genève',
  'lat': 46.19590759277344,
  'lon': 6.136425971984863,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Unknown Amenity',
  'quarter': 'Jonction',
  'city': 'Genève',
  'address': "Rue de l'Ecole-de-Médecine, 1205, Genève",
  'zipcode': '1205'},
 {'label': 'Université de Genève, Valais Campus Sion (VS) - Sion',
  'lat': 46.22985076904297,
  'lon': 7.398563861846924,
  'objectclass': 'TLM_NUTZUNGSAREAL',
  'Kategorie': 'UH',
  'schoolshort': 'UNIGE',
  'name': 'Fondation universitaire Kurt B

In [54]:
import datetime

# Aktuelles Datum für den Dateinamen
current_date = datetime.date.today()
filename = f'colleges_revisioned_{current_date}.json'

# Speichern der Daten in der Datei
with open(filename, 'w') as file:
    json.dump(filtered_data_uzh, file)